In [2]:
import json
import pandas as pd
import re

In [3]:
with open('../dataset_path.json', 'r') as f:
    paths = json.load(f)

raw_data_path = paths['movielense']['raw']
processed_data_path = paths['movielense']['processed']

In [4]:
interaction_df = pd.read_csv(f'{raw_data_path}/ratings.dat', sep='::', engine='python', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

item_feature_df = pd.read_csv(f'{raw_data_path}/movies.dat', sep='::', engine='python', header=None, names=['item_id', 'movie_title', 'genres'], encoding='ISO-8859-1') 

genres = set()

for genre in item_feature_df['genres']:
    genre_list = genre.split('|')
    genres.update(genre_list)

genres = list(genres)

for col in genres:
    item_feature_df[col] = [0] * len(item_feature_df)

for row in item_feature_df.itertuples():
    idx = row.Index
    for genre in row.genres.split('|'):
        item_feature_df.loc[idx, genre] = 1   

item_feature_df.drop(['genres'], axis=1, inplace=True)
item_feature_df.tail()


titles = []
years = []
for title_year in item_feature_df['movie_title']:
    # Extract year assuming it is in parentheses at the end
    year = re.findall(r'\((\d{4})\)', title_year)
    year = year[0] if year else None

    # Extract title: everything before the year and parentheses
    title = re.findall(r'^(.*)\s\(\d{4}\)', title_year)
    title = title[0] if title else title_year
    
    titles.append(title)
    years.append(year)

item_feature_df['title'] = titles
item_feature_df['year'] = years
item_feature_df['year'] = item_feature_df['year'].astype(int)

item_feature_df.drop(['movie_title'], axis=1, inplace=True)

In [5]:
item_feature_df

,item_id,Documentary,Crime,Sci-Fi,Mystery,Musical,Western,Drama,Film-Noir,Comedy,...,Fantasy,Action,Children's,War,Romance,Animation,Thriller,Adventure,title,year
0,1,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,Toy Story,1995
1,2,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,Jumanji,1995
2,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,Grumpier Old Men,1995
3,4,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,Waiting to Exhale,1995
4,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Father of the Bride Part II,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Meet the Parents,2000
3879,3949,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,Requiem for a Dream,2000
3880,3950,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,Tigerland,2000
3881,3951,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,Two Family House,2000


In [10]:
interaction_df = pd.read_csv(f'{raw_data_path}/ratings.dat', sep='::', engine='python', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

interaction_df.drop(['timestamp'], axis=1, inplace=True)
len(interaction_df)

1000209

user_id
4775      20
2714      20
160       20
4749      20
4398      20
        ... 
1181    1521
1941    1595
4277    1743
1680    1850
4169    2314
Name: count, Length: 6040, dtype: int64